In [ ]:
import os
env_path = os.path.expanduser("~/my_hw4_env")
print(f"Creating environment at: {env_path} ...")

!python -m venv {env_path}
print("Installing libraries...")
!{env_path}/bin/pip install torch torchvision numpy Pillow opencv-python matplotlib tqdm wandb ipykernel
print("Registering kernel...")
!{env_path}/bin/python -m ipykernel install --user --name=my_hw4_kernel --display-name "My HW4 Kernel"

print("\nSUCCESS! Now refresh the page and change your kernel.")

Creating environment at: /jet/home/rnagaraj/my_hw4_env ...
Installing libraries...

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: /jet/home/rnagaraj/my_hw4_env/bin/python -m pip install --upgrade pip
Registering kernel...
Installed kernelspec my_hw4_kernel in /jet/home/rnagaraj/.local/share/jupyter/kernels/my_hw4_kernel

SUCCESS! Now refresh the page and change your kernel.


In [7]:
!pip install torch torchvision numpy Pillow opencv-python matplotlib tqdm wandb

  Using cached torchvision-0.24.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (5.9 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
  Using cached torch-2.9.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cufft_cu12-11.3.3.83-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.

In [2]:
"""
Pix2Pix Implementation for PhysicsGen Motion Prediction
"""

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
from PIL import Image
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from pathlib import Path
import itertools

try:
    import wandb
except ImportError:
    print("Installing wandb...")
    !pip install wandb -q
    import wandb

In [3]:
os.environ['WANDB_API_KEY'] = 'c0568591bf53551942de5b598dcf8626dae19df1'
wandb.login()

wandb: Currently logged in as: rishin (rishin-carnegie-mellon-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
!ls

HW2P2_final_submission.zip  kaggle_data.json
HW3P2_final_submission.zip  model_arch.txt
HW3P2_v10.ipynb		    model_metadata_2025-10-12_18-23.json
IDL-hw4			    model_metadata_2025-10-12_18-49.json
README.txt		    model_metadata_2025-10-31_15-12.json
acknowledgement.txt	    model_metadata_2025-11-09_19-09.json
bounce_ball.zip		    model_metadata_2025-11-09_19-21.json
bounce_ball_data	    my_hw4_env
checkpoint		    ondemand
checkpoint_p3_new	    submission.csv
checkpoint_submission.json  verification_early_submission.csv
checkpoints		    wandb
config.yaml		    wandb_top_runs.pkl


In [4]:
class CombinedImageDataset(Dataset):
    def __init__(self, data_dir, transform=None, split_direction='horizontal'):
        self.data_dir = Path(data_dir)
        self.transform = transform
        self.split_direction = split_direction
        self.image_files = sorted([
            f for f in os.listdir(self.data_dir) 
            if f.endswith(('.png', '.jpg', '.jpeg'))
        ])
        
        print(f"{len(self.image_files)} combined images from {data_dir}")
        
        if self.image_files:
            first_img = Image.open(self.data_dir / self.image_files[0])
            w, h = first_img.size
            print(f"  Image size: {w}x{h}")
            
            if split_direction == 'horizontal':
                if w > h * 1.5:
                    print(f"  Will split into: {w//2}x{h} (left) and {w//2}x{h} (right)")
                else:
                    print(f"  Image might not be horizontally combined")
            else:
                if h > w * 1.5:
                    print(f"  Will split into: {w}x{h//2} (top) and {w}x{h//2} (bottom)")
                else:
                    print(f"  Image might not be vertically combined")
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_path = self.data_dir / self.image_files[idx]
        combined_img = Image.open(img_path).convert('RGB')
        
        w, h = combined_img.size
        
        if self.split_direction == 'horizontal':
            mid = w // 2
            input_img = combined_img.crop((0, 0, mid, h))
            target_img = combined_img.crop((mid, 0, w, h))
        else:
            mid = h // 2
            input_img = combined_img.crop((0, 0, w, mid))
            target_img = combined_img.crop((0, mid, w, h))
        
        if self.transform:
            input_img = self.transform(input_img)
            target_img = self.transform(target_img)
        
        return input_img, target_img

In [5]:
def create_data_loaders(data_root, batch_size=18, image_size=256, num_workers=2, 
                       split_direction='horizontal'):
    
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    
    print(f"Split direction: {split_direction}")
    
    
    train_path = Path(data_root) / 'train_double'
    print("\nTrain set:")
    train_dataset = CombinedImageDataset(
        str(train_path), 
        transform=transform,
        split_direction=split_direction
    )
    
    val_path = Path(data_root) / 'val_double'
    test_path = Path(data_root) / 'test_double'
    
    if val_path.exists():
        print("\nValidation set:")
        val_dataset = CombinedImageDataset(
            str(val_path), 
            transform=transform,
            split_direction=split_direction
        )
    else:
        print("\nValidation set: Creating 10% split from training")
        train_size = int(0.9 * len(train_dataset))
        val_size = len(train_dataset) - train_size
        train_dataset, val_dataset = torch.utils.data.random_split(
            train_dataset, [train_size, val_size]
        )
    
    if test_path.exists():
        print("\nTest set:")
        test_dataset = CombinedImageDataset(
            str(test_path), 
            transform=transform,
            split_direction=split_direction
        )
    else:
        print("\nTest set: Using val set")
        test_dataset = val_dataset
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                             shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, 
                           shuffle=False, num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, 
                            shuffle=False, num_workers=num_workers)
    
    return train_loader, val_loader, test_loader

In [6]:
class UNetDown(nn.Module):
    def __init__(self, in_channels, out_channels, normalize=True, dropout=0.0):
        super(UNetDown, self).__init__()
        layers = [nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False)]
        if normalize:
            layers.append(nn.InstanceNorm2d(out_channels, affine=True))
        layers.append(nn.LeakyReLU(0.2))
        if dropout:
            layers.append(nn.Dropout(dropout))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


class UNetUp(nn.Module):
    def __init__(self, in_channels, out_channels, dropout=0.0):
        super(UNetUp, self).__init__()
        layers = [
            nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(out_channels, affine=True),
            nn.ReLU(inplace=True),
        ]
        if dropout:
            layers.append(nn.Dropout(dropout))
        self.model = nn.Sequential(*layers)

    def forward(self, x, skip_input):
        x = self.model(x)
        x = torch.cat((x, skip_input), 1)
        return x

In [7]:
class Generator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(Generator, self).__init__()
        
        self.down1 = UNetDown(in_channels, 64, normalize=False) 
        self.down2 = UNetDown(64, 128)                          
        self.down3 = UNetDown(128, 256)                       
        self.down4 = UNetDown(256, 512)          
        self.down5 = UNetDown(512, 512)
        self.down6 = UNetDown(512, 512)
        self.down7 = UNetDown(512, 512)
        self.down8 = UNetDown(512, 512, normalize=False) 

        
        self.up1 = UNetUp(512, 512, dropout=0.5)
        self.up2 = UNetUp(1024, 512, dropout=0.5)
        self.up3 = UNetUp(1024, 512, dropout=0.5) 
        self.up4 = UNetUp(1024, 512)
        self.up5 = UNetUp(1024, 256)
        self.up6 = UNetUp(512, 128) 
        self.up7 = UNetUp(256, 64)

        self.final = nn.Sequential(
            nn.ConvTranspose2d(128, out_channels, 4, 2, 1),
            nn.Tanh(),
        )

    def forward(self, x):
        
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        d8 = self.down8(d7)

        
        u1 = self.up1(d8, d7)
        u2 = self.up2(u1, d6)
        u3 = self.up3(u2, d5)
        u4 = self.up4(u3, d4)
        u5 = self.up5(u4, d3)
        u6 = self.up6(u5, d2)
        u7 = self.up7(u6, d1)

        return self.final(u7)

In [8]:
class Discriminator(nn.Module):
    
    def __init__(self, in_channels=6):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, normalization=True):
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalization:
                layers.append(nn.InstanceNorm2d(out_filters, affine=True))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(in_channels, 64, normalization=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1, bias=False)
        )

    def forward(self, img_A, img_B):
        img_input = torch.cat((img_A, img_B), 1)
        return self.model(img_input)

In [9]:
def compute_gradient_penalty(discriminator, real_A, real_B, fake_B, device, lambda_gp=10):
    
    batch_size = real_B.size(0)
    alpha = torch.rand(batch_size, 1, 1, 1, device=device)
    
    interpolates = (alpha * real_B + (1 - alpha) * fake_B).requires_grad_(True)
    d_interpolates = discriminator(real_A, interpolates)
    fake = torch.ones(d_interpolates.size(), device=device, requires_grad=False)
    
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * lambda_gp
    
    return gradient_penalty

In [34]:
def train_epoch(generator, discriminator, train_loader, optimizer_G, optimizer_D, 
                criterion_GAN, criterion_L1, lambda_L1, lambda_gp, device, epoch):
    generator.train()
    discriminator.train()
    
    epoch_g_loss = 0
    epoch_d_loss = 0
    epoch_gan_loss = 0
    epoch_l1_loss = 0
    epoch_gp_loss = 0 
    
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1} - Training')
    for batch_idx, (real_A, real_B) in enumerate(pbar):
        real_A = real_A.to(device)
        real_B = real_B.to(device)
        
        batch_size = real_A.size(0)
        valid = torch.ones((batch_size, 1, 16, 16), device=device, requires_grad=False)
        fake = torch.zeros((batch_size, 1, 16, 16), device=device, requires_grad=False)
        
        optimizer_G.zero_grad()
        fake_B = generator(real_A)
        pred_fake = discriminator(real_A, fake_B)
        loss_GAN = criterion_GAN(pred_fake, valid)
        loss_L1 = criterion_L1(fake_B, real_B)
        loss_G = loss_GAN + lambda_L1 * loss_L1
        loss_G.backward()
        optimizer_G.step()
        
        optimizer_D.zero_grad()
        
        pred_real = discriminator(real_A, real_B)
        loss_real = criterion_GAN(pred_real, valid)
        
        pred_fake = discriminator(real_A, fake_B.detach())
        loss_fake = criterion_GAN(pred_fake, fake)
        
        gp = compute_gradient_penalty(discriminator, real_A, real_B, 
                                     fake_B.detach(), device, lambda_gp)
        
        loss_D = 0.5 * (loss_real + loss_fake) + gp
        loss_D.backward()
        optimizer_D.step()
        
        epoch_g_loss += loss_G.item()
        epoch_d_loss += loss_D.item()
        epoch_gan_loss += loss_GAN.item()
        epoch_l1_loss += loss_L1.item()
        epoch_gp_loss += gp.item()

        if batch_idx % 10 == 0:
            wandb.log({
                'batch/g_loss': loss_G.item(),
                'batch/d_loss': loss_D.item(),
                'batch/gan_loss': loss_GAN.item(),
                'batch/l1_loss': loss_L1.item(),
                'batch/gp_loss': gp.item(),
                'batch/step': epoch * len(train_loader) + batch_idx
            })
        
        pbar.set_postfix({
            'G': f'{loss_G.item():.4f}',
            'D': f'{loss_D.item():.4f}',
            'L1': f'{loss_L1.item():.4f}',
            'GP': f'{gp.item():.4f}'
        })
    
    num_batches = len(train_loader)
    return (epoch_g_loss/num_batches, epoch_d_loss/num_batches,
            epoch_gan_loss/num_batches, epoch_l1_loss/num_batches,
            epoch_gp_loss/num_batches)

In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm
import wandb

def calculate_roundness_loss(prediction):
    """
    Calculates L_roundness: Penalty for geometric distortion.
    
    Note: Since the exact implementation depends on the specific paper's
    definition (e.g., Hough transform differentiability or radial symmetry), 
    this uses a variance-based approach suitable for 'ball' shapes.
    
    It calculates the center of mass and penalizes variance in the 
    distance of activated pixels from that center.
    """
    batch_size, _, h, w = prediction.size()
    y_grid, x_grid = torch.meshgrid(torch.arange(h), torch.arange(w), indexing='ij')
    y_grid, x_grid = y_grid.to(prediction.device).float(), x_grid.to(prediction.device).float()
    pred_sum = torch.sum(prediction, dim=[2, 3], keepdim=True) + 1e-8
    pred_norm = prediction / pred_sum
    com_x = torch.sum(pred_norm * x_grid, dim=[2, 3], keepdim=True)
    com_y = torch.sum(pred_norm * y_grid, dim=[2, 3], keepdim=True)
    dist_sq = (x_grid - com_x)**2 + (y_grid - com_y)**2
    weighted_dist_variance = torch.sum(pred_norm * dist_sq, dim=[2, 3])
    
    return torch.mean(weighted_dist_variance)

def train_epoch_tpl(generator, discriminator, train_loader, optimizer_G, optimizer_D, 
                    criterion_GAN, lambda1, lambda2, lambda3, lambda_gp, device, epoch):
    
    generator.train()
    discriminator.train()
    metrics = {
        'g_loss': 0, 'd_loss': 0, 'gan_loss': 0, 
        'mse_loss': 0, 'round_loss': 0, 'gp_loss': 0
    }
    criterion_MSE = nn.MSELoss() 
    
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1} - TPL Training')
    
    for batch_idx, (real_A, real_B) in enumerate(pbar):
        real_A = real_A.to(device)
        real_B = real_B.to(device)
        
        batch_size = real_A.size(0)
        optimizer_D.zero_grad()
        fake_B = generator(real_A)
        valid = torch.ones((batch_size, 1, 16, 16), device=device, requires_grad=False)
        fake = torch.zeros((batch_size, 1, 16, 16), device=device, requires_grad=False)
        
        pred_real = discriminator(real_A, real_B)
        loss_real = criterion_GAN(pred_real, valid)
        pred_fake = discriminator(real_A, fake_B.detach())
        loss_fake = criterion_GAN(pred_fake, fake)
        gp = compute_gradient_penalty(discriminator, real_A, real_B, 
                                      fake_B.detach(), device, lambda_gp)
        
        loss_D = 0.5 * (loss_real + loss_fake) + gp
        loss_D.backward()
        optimizer_D.step()
        optimizer_G.zero_grad()
        pred_fake_G = discriminator(real_A, fake_B) 
        loss_BCE_valid = criterion_GAN(pred_fake_G, valid)
        loss_MSE = criterion_MSE(fake_B, real_B)
        loss_Roundness = calculate_roundness_loss(fake_B)
        loss_G = (lambda1 * loss_MSE) + \
                 (lambda2 * loss_Roundness) + \
                 (lambda3 * loss_BCE_valid)
                 
        loss_G.backward()
        optimizer_G.step()
        metrics['g_loss'] += loss_G.item()
        metrics['d_loss'] += loss_D.item()
        metrics['gan_loss'] += loss_BCE_valid.item()
        metrics['mse_loss'] += loss_MSE.item()
        metrics['round_loss'] += loss_Roundness.item()
        metrics['gp_loss'] += gp.item()

        if batch_idx % 10 == 0:
            wandb.log({
                'batch/g_loss': loss_G.item(),
                'batch/d_loss': loss_D.item(),
                'batch/gan_bce_loss': loss_BCE_valid.item(),
                'batch/mse_loss': loss_MSE.item(),
                'batch/round_loss': loss_Roundness.item(),
                'batch/step': epoch * len(train_loader) + batch_idx
            })
        
        pbar.set_postfix({
            'G': f'{loss_G.item():.4f}',
            'MSE': f'{loss_MSE.item():.4f}',
            'Round': f'{loss_Roundness.item():.4f}'
        })
    
    num_batches = len(train_loader)
    return {k: v / num_batches for k, v in metrics.items()}

In [11]:
def validate(generator, discriminator, val_loader, criterion_GAN, 
             criterion_L1, lambda_L1, device, epoch):
    generator.eval()
    discriminator.eval()
    
    val_g_loss = 0
    val_d_loss = 0
    
    with torch.no_grad():
        for real_A, real_B in tqdm(val_loader, desc=f'Epoch {epoch+1} - Validation'):
            real_A = real_A.to(device)
            real_B = real_B.to(device)
            
            batch_size = real_A.size(0)
            valid = torch.ones((batch_size, 1, 16, 16), device=device)
            fake = torch.zeros((batch_size, 1, 16, 16), device=device)
            
            fake_B = generator(real_A)
            pred_fake = discriminator(real_A, fake_B)
            loss_GAN = criterion_GAN(pred_fake, valid)
            loss_L1 = criterion_L1(fake_B, real_B)
            loss_G = loss_GAN + lambda_L1 * loss_L1
            
            pred_real = discriminator(real_A, real_B)
            loss_real = criterion_GAN(pred_real, valid)
            pred_fake = discriminator(real_A, fake_B)
            loss_fake = criterion_GAN(pred_fake, fake)
            loss_D = 0.5 * (loss_real + loss_fake)
            
            val_g_loss += loss_G.item()
            val_d_loss += loss_D.item()
    
    num_batches = len(val_loader)
    return val_g_loss/num_batches, val_d_loss/num_batches

In [ ]:
def train_pix2pix(generator, discriminator, train_loader, val_loader, 
                  config, device, save_dir='checkpoints'):
    os.makedirs(save_dir, exist_ok=True)
    
    optimizer_G = optim.Adam(generator.parameters(), 
                            lr=config['lr_generator'], 
                            betas=(0.5, 0.999))
    optimizer_D = optim.Adam(discriminator.parameters(), 
                            lr=config['lr_discriminator'], 
                            betas=(0.5, 0.999))
    
    criterion_GAN = nn.MSELoss()
    criterion_L1 = nn.L1Loss()
    
    best_val_loss = float('inf')
    print("Starting Pix2Pix Training with Gradient Penalty")
    print("="*70)
    print(f"Lambda L1: {config['lambda_l1']}")
    print(f"Lambda GP: {config['lambda_gp']}") 
    print("="*70)
    
    start_time = time.time()
    
    for epoch in range(config['num_epochs']):
        epoch_start = time.time()
        
        train_g, train_d, train_gan, train_l1, train_gp = train_epoch(
            generator, discriminator, train_loader,
            optimizer_G, optimizer_D,
            criterion_GAN, criterion_L1, 
            config['lambda_l1'], config['lambda_gp'], 
            device, epoch
        )
        val_g, val_d = validate(
            generator, discriminator, val_loader,
            criterion_GAN, criterion_L1, config['lambda_l1'],
            device, epoch
        )
        
        epoch_time = time.time() - epoch_start
        wandb.log({
            'epoch': epoch + 1,
            'train/g_loss': train_g,
            'train/d_loss': train_d,
            'train/gan_loss': train_gan,
            'train/l1_loss': train_l1,
            'train/gp_loss': train_gp,  
            'val/g_loss': val_g,
            'val/d_loss': val_d,
            'epoch_time': epoch_time
        })
        
        print(f"\nEpoch [{epoch+1}/{config['num_epochs']}] | Time: {epoch_time:.1f}s")
        print(f"  Train - G: {train_g:.4f}, D: {train_d:.4f}, GP: {train_gp:.4f}")
        print(f"  Val   - G: {val_g:.4f}, D: {val_d:.4f}")
        
        if val_g < best_val_loss:
            best_val_loss = val_g
            checkpoint = {
                'epoch': epoch,
                'generator_state_dict': generator.state_dict(),
                'discriminator_state_dict': discriminator.state_dict(),
                'val_loss': val_g,
            }
            torch.save(checkpoint, os.path.join(save_dir, 'best_pix2pix.pth'))
            wandb.save(os.path.join(save_dir, 'best_pix2pix.pth'))
            print(f"  ✓ Saved best model")
        if (epoch + 1) % 10 == 0:
            visualize_results(generator, val_loader, device, epoch)
    
    total_time = time.time() - start_time
    wandb.run.summary['best_val_loss'] = best_val_loss
    wandb.run.summary['total_training_time'] = total_time
    
    print("\n" + "="*70)
    print(f"Training completed in {total_time/3600:.2f} hours")
    print("="*70)

In [ ]:
def train_pix2pix(generator, discriminator, train_loader, val_loader, config, device):
    optimizer_G = torch.optim.Adam(
        generator.parameters(), 
        lr=config['lr_generator'], 
        betas=(0.5, 0.999)
    )
    optimizer_D = torch.optim.Adam(
        discriminator.parameters(), 
        lr=config['lr_discriminator'], 
        betas=(0.5, 0.999)
    )
    criterion_GAN = nn.BCEWithLogitsLoss()
    
    for epoch in range(config['num_epochs']):
        epoch_start = time.time()
        metrics = train_epoch_tpl(
            generator, discriminator, train_loader, 
            optimizer_G, optimizer_D,
            criterion_GAN, 
            config['lambda1'], config['lambda2'], config['lambda3'], 
            config['lambda_gp'], 
            device, epoch
        )
        
        epoch_duration = time.time() - epoch_start
        
        print(f"Epoch {epoch+1}/{config['num_epochs']} "
              f"[{epoch_duration:.0f}s] "
              f"G_Loss: {metrics['g_loss']:.4f} "
              f"MSE: {metrics['mse_loss']:.4f} "
              f"Round: {metrics['round_loss']:.4f}")
        if (epoch + 1) % 5 == 0:
            torch.save(generator.state_dict(), f"generator_epoch_{epoch+1}.pth")
            torch.save(discriminator.state_dict(), f"discriminator_epoch_{epoch+1}.pth")

    print("Training Complete")

In [13]:
def visualize_results(generator, val_loader, device, epoch):
    generator.eval()
    
    with torch.no_grad():
        real_A, real_B = next(iter(val_loader))
        real_A = real_A.to(device)
        fake_B = generator(real_A)
        
        def denorm(x):
            return (x + 1) / 2
        
        num_vis = min(4, len(real_A))
        fig, axes = plt.subplots(3, num_vis, figsize=(4*num_vis, 12))
        
        for i in range(num_vis):
            input_img = denorm(real_A[i]).cpu().permute(1, 2, 0).numpy()
            axes[0, i].imshow(input_img)
            axes[0, i].set_title(f'Input {i+1}')
            axes[0, i].axis('off')
            
            gen_img = denorm(fake_B[i]).cpu().permute(1, 2, 0).numpy()
            axes[1, i].imshow(gen_img)
            axes[1, i].set_title(f'Generated {i+1}')
            axes[1, i].axis('off')
            
            target_img = denorm(real_B[i]).cpu().permute(1, 2, 0).numpy()
            axes[2, i].imshow(target_img)
            axes[2, i].set_title(f'Ground Truth {i+1}')
            axes[2, i].axis('off')
        
        plt.tight_layout()
        wandb.log({f"generations/epoch_{epoch+1}": wandb.Image(fig), "epoch": epoch + 1})
        plt.close()

In [14]:
class BallDetector:
    
    def __init__(self, ball_radius=15, min_radius=10, max_radius=25):
        self.ball_radius = ball_radius
        self.min_radius = min_radius
        self.max_radius = max_radius
    
    def detect_ball(self, image):
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image
        
        blurred = cv2.GaussianBlur(gray, (9, 9), 2)
        circles = cv2.HoughCircles(
            blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=50,
            param1=50, param2=30,
            minRadius=self.min_radius, maxRadius=self.max_radius
        )
        
        if circles is None or len(circles[0]) == 0:
            return None
        
        circle = circles[0][0]
        return {'position': (circle[0], circle[1]), 'radius': circle[2]}

In [15]:
def evaluate_pix2pix(generator, test_loader, device):
    generator.eval()
    detector = BallDetector()
    
    all_errors_x = []
    all_errors_y = []
    failed_predictions = 0
    total_predictions = 0
    sample_predictions = []
    
    print("Evaluating Pix2Pix Model")
    
    with torch.no_grad():
        for batch_idx, (real_A, real_B) in enumerate(tqdm(test_loader, desc='Evaluating')):
            real_A = real_A.to(device)
            fake_B = generator(real_A)
            
            fake_imgs = ((fake_B + 1) / 2 * 255).cpu().numpy().astype(np.uint8)
            real_imgs = ((real_B + 1) / 2 * 255).numpy().astype(np.uint8)
            
            for i in range(len(fake_imgs)):
                total_predictions += 1
                
                pred = fake_imgs[i].transpose(1, 2, 0)
                target = real_imgs[i].transpose(1, 2, 0)
                
                # Log first 10 samples
                if len(sample_predictions) < 10:
                    fig, axes = plt.subplots(1, 2, figsize=(8, 4))
                    axes[0].imshow(pred)
                    axes[0].set_title('Prediction')
                    axes[0].axis('off')
                    axes[1].imshow(target)
                    axes[1].set_title('Ground Truth')
                    axes[1].axis('off')
                    plt.tight_layout()
                    sample_predictions.append(wandb.Image(fig))
                    plt.close()
                
                pred_ball = detector.detect_ball(pred)
                target_ball = detector.detect_ball(target)
                
                if pred_ball is None or target_ball is None:
                    failed_predictions += 1
                    continue
                
                pos_error_x = abs(pred_ball['position'][0] - target_ball['position'][0])
                pos_error_y = abs(pred_ball['position'][1] - target_ball['position'][1])
                
                all_errors_x.append(pos_error_x)
                all_errors_y.append(pos_error_y)
    
    failure_rate = (failed_predictions / total_predictions) * 100
    
    results = {
        'test/position_x_mean': np.mean(all_errors_x) if len(all_errors_x) > 0 else 0,
        'test/position_x_std': np.std(all_errors_x) if len(all_errors_x) > 0 else 0,
        'test/position_y_mean': np.mean(all_errors_y) if len(all_errors_y) > 0 else 0,
        'test/position_y_std': np.std(all_errors_y) if len(all_errors_y) > 0 else 0,
        'test/failure_rate': failure_rate,
        'test/total_predictions': total_predictions,
        'test/failed_predictions': failed_predictions
    }
    
    wandb.log(results)
    wandb.log({"test/sample_predictions": sample_predictions})
    
    comparison_table = wandb.Table(
        columns=["Metric", "Paper (Pix2Pix)", "Our Results"],
        data=[
            ["Position X Error (pixels)", "6.28 ± 7.98", f"{results['test/position_x_mean']:.2f} ± {results['test/position_x_std']:.2f}"],
            ["Position Y Error (pixels)", "11.7 ± 12.8", f"{results['test/position_y_mean']:.2f} ± {results['test/position_y_std']:.2f}"],
            ["Failure Rate (%)", "7%", f"{failure_rate:.2f}%"]
        ]
    )
    wandb.log({"comparison_with_paper": comparison_table})
    
    print("\n" + "="*70)
    print("EVALUATION RESULTS")
    print(f"Total: {total_predictions}, Failed: {failed_predictions} ({failure_rate:.2f}%)")
    
    if len(all_errors_x) > 0:
        print(f"\nPosition X Error: {results['test/position_x_mean']:.2f} ± {results['test/position_x_std']:.2f} pixels")
        print(f"Position Y Error: {results['test/position_y_mean']:.2f} ± {results['test/position_y_std']:.2f} pixels")
    
    print("\n" + "="*70)
    print("COMPARISON WITH PAPER RESULTS")
    print("Paper Pix2Pix Results:")
    print("  Position X: 6.28 ± 7.98 pixels")
    print("  Position Y: 11.7 ± 12.8 pixels")
    print("  Rotation: 17.2 ± 20.8 degrees")
    print("  Roundness: 0.56 ± 0.14 pixels")
    print("  Failure Rate: 7%")
    print("  Number of balls: 93% single ball, 7% no/multiple balls")
    
    return results

In [6]:
import os
print(os.path.abspath("bounce_ball_data"))

/jet/home/rnagaraj/bounce_ball_data


In [31]:
def main(use_wandb=True, wandb_project="physicsgen-pix2pix", wandb_entity=None):
    DATA_ROOT = 'bounce_ball_data/bounce_ball'
    
    CONFIG = {
        'batch_size': 18,
        'lr_discriminator': 1e-4, #try changing to 5e-5
        'lr_generator': 2e-4,
        'num_epochs': 50,
        'lambda_l1': 100,
        'lambda_gp': 10,
        'image_size': 256,
        'num_workers': 2,
        'split_direction': 'horizontal',
    }
    
    if use_wandb:
        wandb.init(
            project=wandb_project,
            entity=wandb_entity,
            config=CONFIG,
            name=f"pix2pix-instancenorm-gp",
            tags=['pix2pix', 'instancenorm', 'gradient-penalty', 'physicsgen']
        )
        config = wandb.config
    else:
        config = CONFIG
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    train_loader, val_loader, test_loader = create_data_loaders(
        DATA_ROOT,
        batch_size=config['batch_size'],
        image_size=config['image_size'],
        num_workers=config['num_workers'],
        split_direction=config['split_direction']
    )
    
    generator = Generator().to(device)
    discriminator = Discriminator().to(device)
    print(f"\nGenerator params: {sum(p.numel() for p in generator.parameters()):,}")
    print(f"Discriminator params: {sum(p.numel() for p in discriminator.parameters()):,}")

    if use_wandb:
        wandb.watch(generator, log='all', log_freq=100)
        wandb.watch(discriminator, log='all', log_freq=100)
    
    train_pix2pix(generator, discriminator, train_loader, val_loader, config, device)
    
    if use_wandb:
        wandb.finish()
    
    return generator, discriminator

In [ ]:
def main(use_wandb=True, wandb_project="physicsgen-pix2pix", wandb_entity=None):
    DATA_ROOT = 'bounce_ball_data/bounce_ball'
    
    CONFIG = {
        'batch_size': 18,
        'lr_discriminator': 1e-4, 
        'lr_generator': 2e-4,
        'num_epochs': 50,
        'lambda1': 100.0,  
        'lambda2': 10.0, 
        'lambda3': 1.0, 
        # ----------------------------------------------
        
        'lambda_gp': 10,
        'image_size': 256,
        'num_workers': 2,
        'split_direction': 'horizontal',
    }
    
    if use_wandb:
        wandb.init(
            project=wandb_project,
            entity=wandb_entity,
            config=CONFIG,
            name=f"pix2pix-TPL-loss", 
            tags=['pix2pix', 'TPL', 'physicsgen', 'roundness-loss']
        )
        config = wandb.config
    else:
        config = CONFIG
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    train_loader, val_loader, test_loader = create_data_loaders(
        DATA_ROOT,
        batch_size=config['batch_size'],
        image_size=config['image_size'],
        num_workers=config['num_workers'],
        split_direction=config['split_direction']
    )
    generator = Generator().to(device)
    discriminator = Discriminator().to(device) 
    
    print(f"\nGenerator params: {sum(p.numel() for p in generator.parameters()):,}")
    print(f"Discriminator params: {sum(p.numel() for p in discriminator.parameters()):,}")

    if use_wandb:
        wandb.watch(generator, log='all', log_freq=100)
        wandb.watch(discriminator, log='all', log_freq=100)
    train_pix2pix(generator, discriminator, train_loader, val_loader, config, device)
    
    if use_wandb:
        wandb.finish()
    
    return generator, discriminator

In [25]:
!ls 


HW2P2_final_submission.zip  kaggle_data.json
HW3P2_final_submission.zip  model_arch.txt
HW3P2_v10.ipynb		    model_metadata_2025-10-12_18-23.json
IDL-hw4			    model_metadata_2025-10-12_18-49.json
README.txt		    model_metadata_2025-10-31_15-12.json
acknowledgement.txt	    model_metadata_2025-11-09_19-09.json
bounce_ball.zip		    model_metadata_2025-11-09_19-21.json
bounce_ball_data	    my_hw4_env
checkpoint		    ondemand
checkpoint_p3_new	    submission.csv
checkpoint_submission.json  verification_early_submission.csv
checkpoints		    wandb
config.yaml		    wandb_top_runs.pkl


In [33]:
if __name__ == '__main__':
    DATA_ROOT = 'bounce_ball_data/bounce_ball'
    USE_WANDB = True
    WANDB_PROJECT = "physicsgen-pix2pix"
    
    generator, discriminator = main(
        use_wandb=USE_WANDB,
        wandb_project=WANDB_PROJECT
    )

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Using device: cuda
Split direction: horizontal

Train set:
44835 combined images from bounce_ball_data/bounce_ball/train_double
  Image size: 1024x512
  Will split into: 512x512 (left) and 512x512 (right)

Validation set:
58 combined images from bounce_ball_data/bounce_ball/val_double
  Image size: 1024x512
  Will split into: 512x512 (left) and 512x512 (right)

Test set:
1600 combined images from bounce_ball_data/bounce_ball/test_double
  Image size: 1024x512
  Will split into: 512x512 (left) and 512x512 (right)

Generator params: 54,413,955
Discriminator params: 2,769,600


Epoch 1 - TPL Training: 100%|█| 2491/2491 [10:26<00:00,  3.98it/s, G=-335201.2188, MSE=0.8365, Round=-33


Epoch 1/50 [627s] G_Loss: -46610478.6044 MSE: 0.8471 Round: -4661056.5279


Epoch 2 - TPL Training: 100%|█| 2491/2491 [10:20<00:00,  4.01it/s, G=-194331.6875, MSE=1.0814, Round=-19


...

Epoch 34/50 [615s] G_Loss: -217787.6693 MSE: 1.2106 Round: -21790.9998


Epoch 35 - TPL Training:  93%|▉| 2316/2491 [09:40<00:43,  4.01it/s, G=89946.1484, MSE=1.1306, Round=8983

: 

: 

: 